## Einrichtung von Julia JuMP
---
Überprüfen Sie die bereits installierten Pakete. In der JuliaBox sind möglicherweise alle notwendigen Pakete bereits vorhanden.

In [1]:
using Pkg;
Pkg.status()

Status `C:\Users\ICH\.juliapro\JuliaPro_v1.4.1-1\environments\v1.4\Project.toml`
  [c52e3926] Atom v0.12.10 ⚲
  [9961bab8] Cbc v0.7.0
  [e2554f3b] Clp v0.8.0
  [7a1cc6ca] FFTW v1.2.2
  [7073ff75] IJulia v1.21.2
  [4076af6c] JuMP v0.21.2
  [e5e0dc1b] Juno v0.8.1 ⚲
  [4722fa14] PkgAuthentication v0.1.2
  [f3b207a7] StatsPlots v0.14.6
  [44d3d7a6] Weave v0.10.2
  [44cfe95a] Pkg 


Installieren Sie ``JuMP`` , mit dem Sie einfach Optimierungsprogramme formulieren können, sowie ``Cbc``, einen open-source Solver zum Lösen des Problems, und ``StatsPlots`` zur Visualisierung der Lösung.


In [21]:
Pkg.add("JuMP");
Pkg.add("Cbc");

Richten Sie die installierten Pakete so ein, dass sie im folgenden Code verwendet werden können.

In [2]:
using JuMP, Cbc;

# Planspiel Schritt 2: Capacitated Lot Sizing Problem



### Ergebnisse aus Teil 1

In [3]:
F_Aggr = [0.0  0.0  0.0  0.0  0.0  0.0
          0.0  0.0  0.0  0.0  0.0  0.0
          0.0  0.0  0.0  0.0  0.0  0.0];

#Hier ist die Lösung des langfristigen Fremdbezuges aus Teil 1 dargestellt
#Die Zeilen stehen für die Produkte und die Spalten für die Perioden

In [4]:
ZAggr = [21.0  6.0  0.0  0.0  38.0  40.0];

#Hier ist die genutzte Zusatzkapazität pro Periode aus Teil 1 dargestellt

In [5]:
LAggr = [0.0  0.0  0.0  823.0  902.0  647.0  0.0
         0.0  0.0  0.0    0.0    1.0    0.0  0.0
         0.0  0.0  0.0    9.0    3.0    1.0  0.0 ];

#Hier sind die Lagerbestände der Produkte pro Periode aus Teil 1 dargestellt
#Wieso sind hier sieben Spalten dargestellt, obwohl nur sechs Perioden betrachtet werden? 
#Tipp: Wann wird der Lagerbestand gemessen?

## Erstellen Sie das Modell namens ``m`` und geben Sie als zu verwendenden Solver Cbc an.
---

In [3]:
m = Model(Cbc.Optimizer);
set_optimizer_attribute(m, "seconds", 60);

# beschränkt die Rechenzeit für die Optimierung auf 60 Sekunden

### Mengen und Parameter

Fügen Sie die Mengen ein.


In [7]:
#Mengen
Produkte =   ["HIER CODE EINGEBEN"]; 
Perioden =   ["HIER CODE EINGEBEN"]; 
#Hinweis: Erstellen Sie für jeden Monat ein eigenes Modell (mit je 4 Wochen) 
#--> Sie brauchen also sechs seperate Dateien; eine für jeden Monat

Ressourcen = ["HIER CODE EINGEBEN"];


#Längen
I = length(Produkte);
T = length(Perioden);
J = length(Ressourcen);

Fügen Sie die Parameter ein.

In [8]:
#Hinweis: Überlegen Sie welche Parameter von Monats- auf Wochenwerte heruntergerechnet werden müssen

kl = ["HIER CODE EINGEBEN"];    #Lagerkosten der Produkte in Euro pro Woche

ks = ["HIER CODE EINGEBEN"];    #Rüstkostensatz der Produkte

ts = ["HIER CODE EINGEBEN"];    #Rüstzeiten der Produkte

tp = [0.105, 0.095, 0.065];     #Stückbearbeitungszeiten der Produkte

M = 100000;                     #Große Zahl

                   
d = ["HIER CODE EINGEBEN"];  
#Bedarf von Produkt i in Periode t
#=Bsp. für drei Perioden und vier Produkte: d = [900 200 100   
                                                 300 0   69  
                                                 500 30  210
                                                 10  25  60 ]; =#

kf = [320, 432, 240];          #Fremdbezugskosten 1 der Produkte in Euro (Tucher)

kv = [160, 216, 120];          #variable Herstellkosten der Produkte

In [9]:
#Diese Werte sind abhängig von der aggregierten Planung

l_start = ["HIER CODE EINGEBEN"]; #=Anfangslagerbestände des Monats ( =Endbestand Vorperiode, s.h. oben 
                                    'Ergebnisse Teil 1')=#
                                  #Bsp. bei vier Produkten: l_start = [20, 50 ,30 , 10];

l_end = ["HIER CODE EINGEBEN"];   #=Der Endbestand des Monats ist immer der entsprechend berechnete Lagerbestand
                                    aus Schritt 1=#
                                  #Bsp. bei vier Produkten: l_end = [30, 60 ,40 , 5];

c =  ["HIER CODE EINGEBEN"];      #Periodenkapazität
#=Hinweis: Berücksichtigen Sie zusätzlich zur regulären Kapazität auch die in der aggr. Planung ermittelte
Zusatzkapazität aufgeteilt auf 4 Wochen=#

faggr = ["HIER CODE EINGEBEN"]    
                                 #Die wöchentliche Lieferung der bei Cheapress bestellten Produkte
                                 #=Bsp. für drei Perioden und vier Produkte: faagr = [900 200 100   
                                                                                      300 0   69  
                                                                                      500 30  210
                                                                                      10  25  60 ]; =#

0-element Array{Any,1}

### Entscheidungsvariablen
---
Definieren Sie die Entscheidungsvariablen. Achten Sie auf die Definitionsbereiche.

**Nichtnegativitätsbedingung**: Die Entscheidungsvariablen, also die Produktionsmengen, Lagerbestände, Fremdbezugsmengen und die in Anspruch genommenen Zusatzkapazitäten dürfen nur positive Werte annehmen, wir setzen deren Nichtnegativität voraus.

$ \qquad X_{it}, L_{it}, F_{it} \geq 0 \qquad \qquad \forall i \in I, t \in T$


In [10]:
@variables  m begin
            X[1:I,1:T]>=0,Int  #Produktionsmenge von Produkt i in Periode t
            F[1:I,1:T]>=0,Int  #Fremdbezugsmenge von Produkt i in Periode t
            L[1:I,0:T]>=0, Int; #Lagerbestand von Produkt i am Periodenende von t
            end

**Binaerbedingung**: Rüstvorgänge werden immer ganz ($\gamma_{it} = 1$) oder gar nicht ($\gamma_{it} = 0$) ausgeführt. Die binäre Rüstvariable nimmt also entweder den Wert 1 oder 0 an.

 $ \qquad  \gamma_{it} \in \{0,1\}  \qquad\qquad\qquad\qquad\qquad\qquad \forall t \in T, \quad i \in I $

In [11]:
@variable(m,gamma[1:I,1:T],Bin); #Die binäre Rüstvariable von Produkt i in Periode t

## Zielfunktion

**Kostenminimierung:** Der Produktionskosten K sollen minimiert werden. Diese berechnen sich hier aus der Summe der variablen Kosten, der Lagerkosten, der Fremdbezugskosten und Rüstkosten.

$$  \qquad \min K = \displaystyle\sum_{i=1}^I \sum_{t=1}^T (k_i^vX_{it}+k_i^l L_{it}+k_i^fF_{it}+ k^s_i\gamma_{i,t}) $$

In [12]:
@objective(m, Min,  sum(kv[i] * X[i,t] + kl[i] * L[i,t] + kf[i] * F[i,t] + ks[i]*gamma[i,t]
                    for i=1:I for t=1:T));

ErrorException: The objective function `MutableArithmetics.Zero()` is not supported by JuMP.

## Nebenbedingungen

**Kapazitaetsrestriktion:** Die Summe aus Produktions- und Rüstzeit aller Produkte darf in jeder Periode die vorhandene Kapazität der Ressource j nicht überschreiten.

 $$ \qquad \displaystyle\sum_{i=1}^I(t_i^p\cdot X_{it}+t_i^s\cdot\gamma_{it} ) \leq c_t \hspace{40mm} \forall t \in T, j \in J\qquad  $$ 
 


In [13]:
@constraint(m, KapRes[t=1:T, j=1:J], sum(ts[i] * gamma[i,t] + tp[i] * X[i,t] for i=1:I) <= c[t] );

**Lagerbilanzgleichung**: Der Lagerbestand eines Produktes am Ende einer Periode berechnet sich aus der Summe der eingelagerten Menge in der Vorperiode, der Produktionsmenge und der kurzfristigen und langfristigen Fremdbezugsmenge, abzüglich der abgesetzen Menge des Produktes.

$$ \qquad L_{it}=L_{i,t-1}+X_{it}-d_{it}+F_{it}+f_{it}^{aggr} \qquad \forall i \in I , t \in T$$

In [14]:
#Fügen sie den Parameter der langfristigen Fremdbezugsmenge in die Nebenbedingung ein!

@constraint(m, Lager[i=1:I,t=1:T], L[i,t] == L[i,t-1] + X[i,t] + F[i,t] - d[i,t]"HIER CODE EINFUEGEN");

**Anfangslagerbestand**: Der Anfangslagerbestand aller Produkte entspricht dem initial gesetzen $l_{start}$.

**Endlagerbestand**: Der Endlagerbestand aller Produkte entspricht dem initial gesetzen $l_{end}$.
 
  $$ \qquad L_{i,0} = l_{start} \hspace{40mm} \forall  i \in I$$
  
  $$ \qquad L_{i,T} = l_{end} \hspace{40mm} \forall  i \in I$$


In [15]:
@constraint(m, AnfLager[i=1:I], L[i,0] == l_start[i]);


#Fügen Sie eine Nebenbedingung für den Endlagerbestand hinzu

@constraint("HIER CODE EINFUEGEN");

LoadError: In `@constraint()`: Not enough arguments

**Rüstbedingung**: Wenn für ein Produkt in einer Periode nicht gerüstet wird, ist die produzierte Menge dieses Produkts in dieser Periode 0. Wenn für ein Produkt in einer Periode gerüstet wird, wird die produzierte Menge durch die Rüstbedingung nicht eingeschränkt.

 $$ \qquad X_{it} \leq M \cdot \gamma_{it} \hspace{40mm} \forall t \in T, \quad i \in I $$

In [16]:
@constraint(m, Ruestbed[i=1:I,t=1:T], X[i,t] <= M * gamma[i,t]);

## Lösen Sie das Modell.
---

In [17]:
"HIER CODE EINFUEGEN"
#Befehl zum Lösen des Modells einfügen
#Tipp: Im Modell der Vorlesung wurde dieser Befehl bereits verwendet

Lassen Sie sich den Zielfunktionswert Z anzeigen.

In [18]:
cost = "HIER CODE EINFUEGEN"    
#Befehl zum Anzeigen des Zielfunktionswertes einfügen
#Tipp: Im Modell der Vorlesung wurde dieser Befehl bereits verwendet

println("Objective value cost: ", round(Int64,cost)) #gibt den gerundeten Wert der Variable cost aus

UndefVarError: UndefVarError: cost not defined

Platz für weitere Berechnungen: